# APIS EventServer examples

Import modules, reload 'hive' to simplify development of the module:

In [ ]:
import sys
if ('pyprediktoredgeclient' in sys.modules):
    sys.modules.pop('pyprediktoredgeclient')
    print("Pruned module from cache")
sys.path.insert(0, r'C:\tfs\PyPrediktorEdgeClient')
from pyprediktoredgeclient import hive
from pyprediktoredgeclient.util import to_pydatetime
from datetime import datetime, timedelta

from importlib import reload
hive = reload(hive)

Get default APIS HIVE instance, and its EventServer:

In [ ]:
apis = hive.Hive()
es = apis.get_eventserver()

Show event-server configuration:

In [ ]:
cfg = es.get_config()
print("Configuration:")
for k,v in cfg.items():
    print(f"\t{k}={v}")

List all data-types:

In [ ]:
vts = es.get_datatypes()
print("Datatypes:")
for k,v in vts.items():
    print(f"\t{k}: {v.name}")


List all event-types:

In [ ]:
types = es.get_eventtypes()
print("Event types:")
for k,v in types.items():
    print(f"\t{k}: {v.name}, parent={v.parent}, flags={v.flags}")


List all event-fields:

In [ ]:
fields = es.get_eventfields()
print("Event fields:")
for k,v in fields.items():
    t = types[v.eventtype]
    vt = vts[v.vt]
    print(f"\t{k}: {t.name}.{v.name}, vt={vt.name}, flags={v.flags}")

Find a single eventsource with a matching 'browsepath':

In [ ]:
flags = es.browse_flags.Like | es.browse_flags.MatchExact
sources = es.browse('*/AlarmArea/*.Signal198', flags)
assert len(sources) == 1
source = sources[0].Id
for i in sources:
    print(f"{i.Id}: {i.Path}")

Fetch the last N events for a single source from the last X days, filtered on "Message" field:

In [ ]:
import System # Needed for 'System.DateTime.FromFileTime' below, should be fixed...
now = datetime.utcnow()
data = es.query(now, now - timedelta(days=30), source, 0, "Message like '*last was [12]*'", 100)
print(f"Query returned {len(data)} rows for source {source}")
for row in data:
    ts = System.DateTime.FromFileTime(row.Fields[0])
    print(f"{to_pydatetime(ts)} source={row.Fields[3]} type={row.Fields[4]} severity={row.Fields[6]} msg={row.Fields[7]}")